# Gemini

In [7]:
from utils.model_setup import *
from utils.dataset_setup import *
from utils.experiment_setup import *
from utils.save_results import *
from utils.state_control import *
from utils.run_experiment import *

In [8]:
from importlib import reload

In [9]:
model_identifier = 'gemini'
path_to = "./data/"
api_key = 4 # With value -1 it will change every 10 iterations

## Setup

In [10]:
"""
Setup State
"""
use_experiment_state = True

state = create_state(-1, -1, -1, 1, 5, 0)
if use_experiment_state:
  state = retrieve_state(path_to, model_identifier)

{'Model': 1, 'Database': 2, 'Experimento': 6, 'Execucao': 3, 'MaxExecucao': 5, 'Batch': 20}


In [ ]:
while api_key < 100:
    api_key += 1
    try:
        execute_experiment(state, model_identifier, path_to, api_key%5 + 1)
    except Exception as e:
        time.sleep(1800)
        continue


Choosen Model: gemini-1.5-flash


 Setting model gemini-1.5-flash with API Key 1 


genai.GenerativeModel(
    model_name='models/gemini-1.5-flash',
    generation_config={},
    safety_settings={},
    tools=None,
    system_instruction=None,
    cached_content=None
)
Dataset escolhido: aes_enem_dataset
Experimento escolhido: exp6
Create exp6
Prompt template: 
Conheça as cinco competências cobradas pelo Inep na redação do ENEM:

1. Domínio da escrita formal da língua portuguesa

É avaliado se a redação do participante está adequada às regras de ortografia, como acentuação, ortografia, uso de hífen, emprego de letras maiúsculas e minúsculas e separação silábica. Ainda são analisadas a regência verbal e nominal, concordância verbal e nominal, pontuação, paralelismo, emprego de pronomes e crase.

São seis níveis de desempenho: 
200 pontos - Demonstra excelente domínio da modalidade escrita formal da língua portuguesa e de escolha de registro. Desvios gramaticais ou de convenções da escri